## 1. 라이브러리 및 드라이버 등록


In [135]:
from selenium import webdriver 
from bs4 import BeautifulSoup 
import os
from os.path import join, dirname
import pandas as pd

In [2]:
if os.name == 'nt': 
    os_name = 'nt' 
elif os.name == 'mac': 
    os_name = 'mac' 
else: 
    os_name = 'linux'
    
if os_name == 'nt': 
    driver_loc = "C:/Users/KoJiIn/Downloads/chromedriver_win32/chromedriver.exe" ##내 pc에 chroomdrirver 저장해놓은 경로 적어주기
else: 
    driver_loc = "C:/Users/KoJiIn/Downloads/chromedriver_win32/chromedriver.exe"

In [3]:
# 크롬 옵션 설정
option = webdriver.ChromeOptions()
option.add_argument('window-size=1920*1080')

#드라이버 등록
driver = webdriver.Chrome(driver_loc, options=option)

## 2. 키워드 등록

In [4]:
naver_login = 'https://nid.naver.com/nidlogin.login' 
cafe_url = 'https://cafe.naver.com/smartpolytech'

keyword_list = ['스프링', '웹']
result_dic = {}

for keyword in keyword_list :
    result_dic[keyword] = {
        'title' : [],
        'writer' : [],
        'href' : [],
        'date' : []
    }
    
exception_flag = 0 
exception_title_keyword_list = ['하둡','빅데이터'] 
exception_writer_keyword_list = []

In [5]:
result_dic

{'스프링': {'title': [], 'writer': [], 'href': [], 'date': []},
 '웹': {'title': [], 'writer': [], 'href': [], 'date': []}}

## 3. 네이버 로그인

In [15]:
#Login page 
driver.get(naver_login)

In [10]:
#webdriver로 열린 chrome창에 'id'라는 속성을 찾아서 'ID'라는  값을 입력해줌

login_id = 'j_i__'
login_pw =   'ggwldls2341!!'
driver.find_element_by_name('id').send_keys(login_id)
driver.find_element_by_name('pw').send_keys(login_pw)

In [8]:
#위에서 입력한 정보를 토대로 자동으로 Login 버튼을 클릭
#이 방법은 간단하지만 네이버 로그인 정책 바뀐 후로 자동입력 방지 문자를 입력해야한다는 번거로움 생김
driver.find_element_by_css_selector('#frmNIDLogin > fieldset > input').click()

In [16]:
#다른 login 방법(javascript 문법 이용)
driver.execute_script("document.getElementsByName('id')[0].value=\'"+login_id+"\'")
driver.execute_script("document.getElementsByName('pw')[0].value=\'"+login_pw+"\'")

In [17]:
driver.find_element_by_css_selector('#frmNIDLogin > fieldset > input').click()

## 4. 크롤링

In [18]:
#크롤링 할 url 접속
driver.get(cafe_url)
driver.implicitly_wait(3) #웹 지원 load를 위해 3초 기다려주도록

In [19]:
#크롤링 할 검색어 입력창 변수처리
search_input = driver.find_element_by_xpath('//*[@id="topLayerQueryInput"]')

In [20]:
#스크린샷
driver.get_screenshot_as_file('naver_main_headless.png')

True

In [21]:
#search_input 창에 위에서 적은 keyword 입력
search_input.send_keys(keyword)

In [22]:
#검색 버튼 클릭
search = driver.find_element_by_xpath('//*[@id="info-search"]/form/button')
search.click()
driver.implicitly_wait(3)

In [23]:
#iframe으로 이루어진 웹사이트의 frame을 cafe_main으로 맞춰주는 역할
iframe = driver.find_element_by_css_selector('iframe#cafe_main')

In [24]:
driver.switch_to.frame(iframe)

In [25]:
#게시글 보기 옵션 selectbox클릭
show_element = driver.find_element_by_xpath("""//*[@id="listSizeSelectDiv"]/a""") 

In [26]:
show_element.click()

In [27]:
#50개씩 보기 클릭
show_50_element = driver.find_element_by_xpath("""//*[@id="listSizeSelectDiv"]/ul/li[7]/a""") 
show_50_element.click()

In [28]:
req = driver.page_source

In [29]:
html = BeautifulSoup(req,'html.parser')

In [30]:
find_div = html.find("div",{"class":"article-board result-board m-tcol-c"})

In [31]:
find_table = find_div.find("table").find("tbody")

In [32]:
find_table

<tbody>
<tr>
<td class="td_article" colspan="2">
<div class="board-number"><div class="inner_number">430</div></div>
<div class="board-list">
<div class="inner_list">
<a class="article" href="/ArticleRead.nhn?clubid=29316181&amp;page=1&amp;userDisplay=50&amp;inCafeSearch=true&amp;searchBy=0&amp;query=%C0%A5&amp;includeAll=&amp;exclude=&amp;include=&amp;exact=&amp;searchdate=all&amp;media=0&amp;sortBy=date&amp;articleid=430&amp;referrerAllArticles=true" onclick="clickcr(this, 'cfa.atitle','','',event);">
<span class="head">
                                    [공지]
                                </span>
                                

                                
                                
                                    [여름방학공지] 2019년도 2기생 여러분 여름방학 잘 보내시길 바랍니다.
                                    
                                
                            </a>
<span class="list-i-img"><i class="blind">사진</i></span>
<div class="result_contents" name="searchContent" sty

In [91]:
find_tr = find_table.find_all("tr")

In [82]:
find_href = find_tr.find("a",{"class":"article"}).attrs['href']

In [84]:
find_title = find_tr.find("a",{"class":"article"}).text.strip()

In [86]:
find_title

'[공지]\n                                \n                                \n\n                                \n                                \n                                    [여름방학공지] 2019년도 2기생 여러분 여름방학 잘 보내시길 바랍니다.'

In [65]:
find_writer = find_tr.find("div",{"class":"pers_nick_area"})

In [66]:
find_writer.text.strip()

'김효관교수'

In [67]:
find_date = find_tr.find("td",{"class":"td_date"})

In [70]:
find_date.text

'2019.07.15.'

In [132]:
title_list = []
href_list = []
writer_list = []
date_list = []

for tr in find_tr :
    find_title = tr.find_all("a",{"class":"article"})
    find_writer = tr.find_all("div",{"class":"pers_nick_area"})
    find_date = tr.find_all("td",{"class":"td_date"})
    for title in find_title :
        title_list.append(title.text.strip())
        href_list.append(title.attrs['href'])
    for writer in find_writer :
        writer_list.append(writer.text.strip())
    for date in find_date :
        date_list.append(date.text)
        
result_dic['웹']['title'] = title_list
result_dic['웹']['href'] = href_list
result_dic['웹']['writer'] = writer_list
result_dic['웹']['date'] = date_list

In [ ]:
## dict의 value는 중복될 수 있지만 key는 중복될 수 없다..
## 키가 중복되면 마지막에 입력한 값으로 덮어씌워짐...!!!!!
## dict의 단일 수정은 key로 접근, 여러 값 수정은 update 메소드 사용

In [133]:
result_dic

{'스프링': {'title': [], 'writer': [], 'href': [], 'date': []},
 '웹': {'title': ['[공지]\n                                \n                                \n\n                                \n                                \n                                    [여름방학공지] 2019년도 2기생 여러분 여름방학 잘 보내시길 바랍니다.',
   'JAVA/ JSP/ 웹개발 / 소프트웨어개발자 신입',
   '[스프링부트] Session 처리',
   '자바 개발환경설정 (Java_home) 설정 및 시험문제 힌트',
   '[이러닝 공지] 한기대 교육컨텐츠 수강관련 안내',
   '[스마트금융과] 교육생 모집 웹페이지 접속자 분석',
   '[공지] 2019년도 스마트금융과 신입생 모집 (세부내용)',
   '데이터스트림즈 솔루션 매뉴얼 위치',
   '성한빈 면접후기 입니다.',
   '여름방학 및 교수일정 공지',
   'python 웹서버 구축에 대한 라이브러리 설명',
   'MyBatis 연동',
   '05.15 웹UI 강의자료',
   '웹서버 실행 중 200, 403 호출이 동시에 응답할때(파비콘 요청)_업데이트',
   '웹UI중간고사 파일',
   '[공지] 오라클 XE 설치 후 기본 브라우저 포트 변경 (8080 -> 9191)',
   'sql실습, 웹UI프로그래밍 1학기 잔여 강의 일정입니다.',
   '아이폰 프로그래밍 - 웹 연결 어플리케이션',
   'JavaScript 개발환경 설정 (Encoding 에러 포함)',
   '자바 웹 프레임워크 (Spring)'],
  'writer': ['김효관교수',
   '김효관교수',
   '황원용교수',
   '김효관교수',
   '김효관교수',
   '김효관교수',
   '김효관교수',
  

In [137]:
from pandas import DataFrame

In [138]:
result_data = DataFrame(result_dic['웹'])

In [139]:
result_data

,title,writer,href,date
0,[공지]\n \n ...,김효관교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2019.07.15.
1,JAVA/ JSP/ 웹개발 / 소프트웨어개발자 신입,김효관교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2019.07.10.
2,[스프링부트] Session 처리,황원용교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2019.06.28.
3,자바 개발환경설정 (Java_home) 설정 및 시험문제 힌트,김효관교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2019.04.16.
4,[이러닝 공지] 한기대 교육컨텐츠 수강관련 안내,김효관교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2019.03.29.
5,[스마트금융과] 교육생 모집 웹페이지 접속자 분석,김효관교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2019.01.01.
6,[공지] 2019년도 스마트금융과 신입생 모집 (세부내용),김효관교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2018.11.02.
7,데이터스트림즈 솔루션 매뉴얼 위치,김효관교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2018.09.29.
8,성한빈 면접후기 입니다.,2018성한빈,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2018.09.17.
9,여름방학 및 교수일정 공지,김효관교수,/ArticleRead.nhn?clubid=29316181&page=1&userDi...,2018.07.16.
